# Table of Contents
* [Import Data from SQLite Database](#Import-Data-from-SQLite-Database)
* [Feature Engineering](#Feature-Engineering)
* [Aggregate columns at SK_ID_PREV level](#Aggregate-columns-at-SK_ID_PREV-level)
* [Merge and save all 4 data parts](#Merge-and-save-all-4-data-parts)
* [Aggregate columns at SK_ID_CURR level](#Aggregate-columns-at-SK_ID_CURR-level)

## Import Data from SQLite Database

In [1]:
import pandas as pd
import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt
%matplotlib inline  
import statistics
from scipy import stats
from scipy.stats import t
from scipy.stats import norm
import seaborn as sns

In [2]:
import sqlite3
from sqlite3 import Error
import csv
# open the connection to read in the datasets, remember to close the connection at the end of the code
con = sqlite3.connect(r"pythonsqlite.db")
cur = con.cursor()

In [3]:
# read in bureau balance data from sqlite database
sql_stmt = '''SELECT A.* FROM pos_bal_sql as A '''
pos_bal = pd.read_sql(sql_stmt, coerce_float=True, con=con)
# replace field that's entirely space (or empty) with NaN
pos_bal.replace(r'^\s*$', np.nan, regex=True, inplace=True)
# close connection
con.close()

## Feature Engineering

Since NAME_CONTRACT_STATUS has 9 categories and very few counts in some categories, to reduce the number of categories, only keep Active and Completed categories, all other status are consolidated into 1 category called 'Other'. 

In [4]:
# combine all non Active non complete status and define them as category 'Other'.
pos_bal['NAME_CONTRACT_STATUS'][~pos_bal['NAME_CONTRACT_STATUS'].isin(['Active','Completed'])] = 'Other'
pos_bal['NAME_CONTRACT_STATUS'].value_counts()

/Users/lisalb168/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Active       9151119
Completed     744883
Other         105356
Name: NAME_CONTRACT_STATUS, dtype: int64

## Aggregate columns at SK_ID_PREV level

Use the following logic for aggregation: Variable SK_DPD means number of Days past due during each month in the past at each previous and current application level. Since over 95% of the values are 0, and only less than 5% of the values are >= 3, create a new dummy variable to group all the >= 3 values into 1 category. Treatment for SK_DPD_DEF is similar to SK_DPD.
1. NAME_CONTRACT_STATUS: Create a new column to hold number of completed contracts. All others will be considered as non-completed.
2. Only keep the max and min of CNT_INSTALMENT : term of previous credit
3. Only keep the max and min of CNT_INSTALMENT_FUTURE: Installments left to pay on the previous credit
4. SK_DPD: create a new variable 'COUNT_DPD_GE3'
5. SK_DPD_DEF: create a new variable 'COUNT_DPD_DEF_GE0'

In [5]:
# open connection to sqlite database 
con = sqlite3.connect(r"pythonsqlite.db")
cur = con.cursor()

In [6]:
pos_p1 = pos_bal[['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE','NAME_CONTRACT_STATUS']]

# write pos_p1 to sqlite database and save it, if don't have index=False, there will be an extra index column
pos_p1.to_sql(name='pos_p1', index=False, con=con)

# group by curr and prev IDs, define a new column STATUS_COMPLETED to hold number of completed contracts:
sql_sm = '''SELECT SK_ID_CURR,
                SK_ID_PREV,
                MAX(CASE WHEN NAME_CONTRACT_STATUS = 'Completed' THEN 1 ELSE 0 END) AS STATUS_COMPLETED
            FROM pos_p1
            GROUP BY SK_ID_CURR, SK_ID_PREV'''
pos_p1_gp = pd.read_sql(sql_sm, coerce_float=True, con=con)

display(pos_p1_gp.head())
print(pos_p1_gp.shape)

,SK_ID_CURR,SK_ID_PREV,STATUS_COMPLETED
0,100001,1369693,1
1,100001,1851984,1
2,100002,1038818,0
3,100003,1810518,1
4,100003,2396755,0


(936325, 3)


In [7]:
# keep the max and min of CNT_INSTALMENT_FUTURE
pos_p2 = pos_bal[['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE','CNT_INSTALMENT_FUTURE']]

# write pos_p2 to sqlite database and save it, if don't have index=False, there will be an extra index column
pos_p2.to_sql(name='pos_p2', index=False, con=con)

# group by CURR and PREV and keep min and max CNT_INSTALMENT for each unique group
sql_sm = '''SELECT SK_ID_CURR,
                SK_ID_PREV,
                MAX(CNT_INSTALMENT_FUTURE) AS MAX_CNT_INSTALMENT_FUTURE,
                MIN(CNT_INSTALMENT_FUTURE) AS MIN_CNT_INSTALMENT_FUTURE             
            FROM pos_p2
            GROUP BY SK_ID_CURR, SK_ID_PREV'''
pos_p2_gp = pd.read_sql(sql_sm, coerce_float=True, con=con)

display(pos_p2_gp.head())
print(pos_p2_gp.shape)

,SK_ID_CURR,SK_ID_PREV,MAX_CNT_INSTALMENT_FUTURE,MIN_CNT_INSTALMENT_FUTURE
0,100001,1369693,4.0,0.0
1,100001,1851984,2.0,0.0
2,100002,1038818,24.0,6.0
3,100003,1810518,12.0,0.0
4,100003,2396755,12.0,1.0


(936325, 4)


In [8]:
pos_p3 = pos_bal[['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE','CNT_INSTALMENT']]

# write pos_p3 to sqlite database and save it, if don't have index=False, there will be an extra index column
pos_p3.to_sql(name='pos_p3', index=False, con=con)

# group by CURR and PREV and keep min and max CNT_INSTALMENT for each unique group
sql_sm = '''SELECT SK_ID_CURR,
                SK_ID_PREV,
                MAX(CNT_INSTALMENT) AS MAX_CNT_INSTALMENT,
                MIN(CNT_INSTALMENT) AS MIN_CNT_INSTALMENT              
            FROM pos_p3
            GROUP BY SK_ID_CURR, SK_ID_PREV'''
pos_p3_gp = pd.read_sql(sql_sm, coerce_float=True, con=con)

display(pos_p3_gp.head())
print(pos_p3_gp.shape)

,SK_ID_CURR,SK_ID_PREV,MAX_CNT_INSTALMENT,MIN_CNT_INSTALMENT
0,100001,1369693,4.0,4.0
1,100001,1851984,4.0,4.0
2,100002,1038818,24.0,24.0
3,100003,1810518,12.0,7.0
4,100003,2396755,12.0,12.0


(936325, 4)


In [9]:
pos_p4 = pos_bal[['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE','SK_DPD','SK_DPD_DEF']]

# write pos_p4 to sqlite database and save it, if don't have index=False, there will be an extra index column
pos_p4.to_sql(name='pos_p4', index=False, con=con)

# group by CURR and PREV and keep, add a column to track past due days <= 3 and > 3 
sql_sm = '''SELECT SK_ID_CURR,
                SK_ID_PREV,
                SUM(CASE WHEN SK_DPD > 3 THEN 1 ELSE 0 END) AS COUNT_DPD_GE3,
                SUM(CASE WHEN SK_DPD_DEF > 0 THEN 1 ELSE 0 END) AS COUNT_DPD_DEF_GE0              
            FROM pos_p4
            GROUP BY SK_ID_CURR, SK_ID_PREV'''
pos_p4_gp = pd.read_sql(sql_sm, coerce_float=True, con=con)

display(pos_p4_gp.head())
print(pos_p4_gp.shape)

,SK_ID_CURR,SK_ID_PREV,COUNT_DPD_GE3,COUNT_DPD_DEF_GE0
0,100001,1369693,0,0
1,100001,1851984,1,1
2,100002,1038818,0,0
3,100003,1810518,0,0
4,100003,2396755,0,0


(936325, 4)


## Merge and save all 4 data parts

In [10]:
# save the grouped data to database to merge with other parts
pos_p1_gp.to_sql(name='pos_p1_gp', index=False, con=con)
pos_p2_gp.to_sql(name='pos_p2_gp', index=False, con=con)
pos_p3_gp.to_sql(name='pos_p3_gp', index=False, con=con)
pos_p4_gp.to_sql(name='pos_p4_gp', index=False, con=con)

# merge all 4 parts together 
sql_sm = '''SELECT A.*,
                B.MAX_CNT_INSTALMENT_FUTURE,
                B.MIN_CNT_INSTALMENT_FUTURE,
                C.MAX_CNT_INSTALMENT,
                C.MIN_CNT_INSTALMENT,
                D.COUNT_DPD_GE3,
                D.COUNT_DPD_DEF_GE0
            FROM pos_p1_gp AS A 
            INNER JOIN
                 pos_p2_gp AS B ON A.SK_ID_CURR = B.SK_ID_CURR AND A.SK_ID_PREV = B.SK_ID_PREV
            INNER JOIN
                 pos_p3_gp AS C ON A.SK_ID_CURR = C.SK_ID_CURR AND A.SK_ID_PREV = C.SK_ID_PREV
            INNER JOIN
                 pos_p4_gp AS D ON A.SK_ID_CURR = D.SK_ID_CURR AND A.SK_ID_PREV = D.SK_ID_PREV            
            '''
pos_cur_prev_gp = pd.read_sql(sql_sm, coerce_float=True, con=con)

display(pos_cur_prev_gp.head())
print(pos_cur_prev_gp.shape)

# write the aggregated level data to sqlite database and save it, if don't have index=False, there will be an extra index column
pos_cur_prev_gp.to_sql(name='pos_cur_prev_gp', index=False, con=con)

,SK_ID_CURR,SK_ID_PREV,STATUS_COMPLETED,MAX_CNT_INSTALMENT_FUTURE,MIN_CNT_INSTALMENT_FUTURE,MAX_CNT_INSTALMENT,MIN_CNT_INSTALMENT,COUNT_DPD_GE3,COUNT_DPD_DEF_GE0
0,100001,1369693,1,4.0,0.0,4.0,4.0,0,0
1,100001,1851984,1,2.0,0.0,4.0,4.0,1,1
2,100002,1038818,0,24.0,6.0,24.0,24.0,0,0
3,100003,1810518,1,12.0,0.0,12.0,7.0,0,0
4,100003,2396755,0,12.0,1.0,12.0,12.0,0,0


(936325, 9)


## Aggregate columns at SK_ID_CURR level

In [11]:
# aggregate at SK_ID_CURR level, added total number of previous application line
sql_sm = '''SELECT SK_ID_CURR,
                COUNT(SK_ID_PREV) AS TOTAL_NUM_PREV_APPS, 
                SUM(STATUS_COMPLETED) AS SUM_STATUS_COMPLETED,
                MAX(MAX_CNT_INSTALMENT_FUTURE) AS MAX_CNT_INSTALMENT_FUTURE,
                MIN(MIN_CNT_INSTALMENT_FUTURE) AS MIN_CNT_INSTALMENT_FUTURE,
                MAX(MAX_CNT_INSTALMENT) AS MAX_CNT_INSTALMENT,
                MIN(MIN_CNT_INSTALMENT) AS MIN_CNT_INSTALMENT,
                SUM(COUNT_DPD_GE3) AS SUM_DPD_GE3,
                SUM(COUNT_DPD_DEF_GE0) AS SUM_DPD_DEF_GE0
            FROM pos_cur_prev_gp
            GROUP BY SK_ID_CURR       
            '''
pos_cur_gp = pd.read_sql(sql_sm, coerce_float=True, con=con)

display(pos_cur_gp.head())
print(pos_cur_gp.shape)

# write the aggregated level data to sqlite database and save it, if don't have index=False, there will be an extra index column
pos_cur_gp.to_sql(name='pos_cur_gp', index=False, con=con)

con.commit()
con.close()

,SK_ID_CURR,TOTAL_NUM_PREV_APPS,SUM_STATUS_COMPLETED,MAX_CNT_INSTALMENT_FUTURE,MIN_CNT_INSTALMENT_FUTURE,MAX_CNT_INSTALMENT,MIN_CNT_INSTALMENT,SUM_DPD_GE3,SUM_DPD_DEF_GE0
0,100001,2,2,4.0,0.0,4.0,4.0,1,1
1,100002,1,0,24.0,6.0,24.0,24.0,0,0
2,100003,3,2,12.0,0.0,12.0,6.0,0,0
3,100004,1,1,4.0,0.0,4.0,3.0,0,0
4,100005,1,1,12.0,0.0,12.0,9.0,0,0


(337252, 9)
